# Objective

Task: Create an absolutely bare-bones data pipeline. Try to build a "good" output out of 2 or more of the found datasets.

In [3]:
import io, requests
import pandas as pd
from datetime import date
from htmldate import find_date

# Functions

### Check whether updates are needed

**Inputs**: 
- URL to the landing page for a csv file that we know has address data for government buildings
- date last pulled

**Output**:
- boolean indicating whether an update is needed

In [68]:
def needs_update(url, date_last_pulled):
    date_last_updated = find_date(url)
    return date_last_updated > date_last_pulled

### Get addresses

**Input**: URL to a csv file that we know has address data for government buildings

**Output**: list of addresses from the file

In [60]:
def get_addresses(url):
    s = requests.get(url).content
    df = pd.read_csv(io.StringIO(s.decode('utf-8')))

    address_col_names = []
    for x in df.columns:
        if 'address' in x.lower():
            address_col_names.append(x)

    for address_col_name in address_col_names:
        print(list(df[address_col_name]))

# Demo

With 3 outputs from the found datasets

In [86]:
# CA Dept. of Tax and Fee Administration Offices
cdtfa_data_url = 'http://data-cdtfa.opendata.arcgis.com/datasets/6b7ab4e55a2d463db9e902c2496c6e37_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D'
cdtfa_landing_url = 'https://data.ca.gov/dataset/cdtfa-offices'

# Directory of conference rooms available in buildings owned by the CA Dept. of General Services
dgs_data_url = 'https://data.ca.gov/dataset/c5a3c577-3d2f-47fa-85f7-7aae00b2adc3/resource/def5d673-e39a-461d-bef1-a417579fd02e/download/dgs-conference-rooms.csv'
dgs_landing_url = 'https://data.ca.gov/dataset/dgs-conference-rooms'

# CA Dept. of Developmental Services Regional Centers
dds_data_url = 'https://data.chhs.ca.gov/dataset/0e68b593-183d-41d0-80e0-7e828d99a22d/resource/369ea9be-077f-423d-b10d-09022e3fda9f/download/regional-center-information.csv'
dds_landing_url = 'https://data.ca.gov/dataset/regional-center-information'


###  Quick look at data contents

In [91]:
s = requests.get(cdtfa_data_url).content
pd.read_csv(io.StringIO(s.decode('utf-8'))).head()

,X,Y,OBJECTID,doid,city,address1,address2,state,zip,phone,fax,directions,comments,type,DISTRICT_OF_LOC,OfficeType
0,-1.324950e+07,4.216651e+06,1,1,Bakersfield,1800 30th St,Suite 380,CA,93301-1922,(661) 395-2880,(661) 395-2588,"https://www.google.com/maps/dir//1800+30th+St,...",Bakersfield,in,DFB,Bakersfield
1,-1.314243e+07,4.011057e+06,2,6,Cerritos,12750 Center Court Drive South,Ste. 400,CA,90703-8594,(562) 356-1102,(562) 402-8503,https://www.google.com/maps/dir//12750+Center+...,Cerritos,in,AA,Cerritos
2,-1.317931e+07,4.026731e+06,3,2,Culver City,5901 Green Valley Circle,Ste. 200,CA,90230-6948,(310) 342-1000,(310) 342-1061,https://www.google.com/maps/dir//5901+Green+Va...,Culver City,in,AS,Culver City
3,-1.286559e+07,3.867842e+06,4,3,El Centro,1550 West Main St.,,CA,92243-2832,(760) 352-3431,(760) 352-8149,https://www.google.com/maps/dir//1550+W+Main+S...,El Centro,in,FHA,El Centro
4,-1.358663e+07,4.618437e+06,5,32,Fairfield,2480 Hilborn Road,Suite 200,CA,94534,(707) 427-4800,(707) 427-4815,https://www.google.com/maps/dir//2480+Hilborn+...,Fairfield,in,JHF,Fairfield


In [92]:
s = requests.get(dgs_data_url).content
pd.read_csv(io.StringIO(s.decode('utf-8'))).head()

,RegionName,BuildingName,Address1,City,State,ZIP,RoomName,RoomComments,Contact,Telephone,...,VideoConferencing,ProjectionScreen,SpeakerSystem,Projector,TeleDataOutlet,WhiteBoard,VCRorDVD,TenantUse,StateAgencyUse,PublicUse
0,Capitol Historic Region,900 N Street,900 N Street,Sacramento,CA,95814,501,NaN,Jack Nolan,916-654-0033,...,N,Y,N,N,Y,Y,N,N,N,N
1,Capitol Historic Region,914 Capitol Mall,914 Capitol Mall,Sacramento,CA,95814,500,NaN,Jack Nolan,916-654-0033,...,N,Y,N,N,Y,N,Y,Y,N,N
2,Capitol Historic Region,915 Capitol Mall,915 Capitol Mall,Sacramento,CA,95814,587,NaN,Luis Hermosillo,916-653-3382,...,N,Y,N,N,Y,N,N,Y,N,N
3,Region II,31 E. Channel St.,31 E. Channel St.,Stockton,CA,95202,A-10,NaN,Building Manager,209-948-7730,...,N,Y,Y,N,N,N,N,Y,Y,N
4,Region II,31 E. Channel St.,31 E. Channel St.,Stockton,CA,95202,Auditorium,NaN,Building Manager,209-948-7730,...,N,Y,Y,N,N,N,N,Y,Y,N


In [93]:
s = requests.get(dds_data_url).content
pd.read_csv(io.StringIO(s.decode('utf-8'))).head()

,Regional Center,Office,Address,Suite,City,State,Zip Code,Telephone,Website,County Served,Los Angeles Health District Served,Location 1
0,Far Northern Regional Center,Field,421 Chestnut Street,NaN,Mt. Shasta,CA,96067,(530) 222-4791,www.farnorthernrc.org,NaN,NaN,421 Chestnut Street\nMt. Shasta 96067\n(41.315...
1,North Bay Regional Center,Main,610 Airpark Road,NaN,Napa,CA,94558,(707) 256-1100,www.nbrc.net,Solano,NaN,"610 Airpark Road\nNapa 94558\n(38.218198, -122..."
2,San Andreas Regional Center,Main,300 Orchard City Drive,Ste. 170,Campbell,CA,95008,(408) 374-9960,www.sarc.org,Santa Clara,NaN,300 Orchard City Drive\nCampbell 95008\n(37.28...
3,Regional Center of Orange County,Mailing,P.O Box 22010,NaN,Santa Ana,CA,92702,(714) 796-5100,www.rcocdd.com,NaN,NaN,P.O Box 22010\nSanta Ana 92702\n
4,Inland Regional Center,Main,1365 South Waterman Avenue,NaN,San Bernardino,CA,92408,(909) 890-3000,www.inlandrc.org,San Bernardino,NaN,1365 South Waterman Avenue\nSan Bernardino 924...


## Check whether the data needs to be pulled again

In [85]:
date_last_pulled = str(date.today()) # Example 1

for landing_url in [cdtfa_landing_url, dgs_landing_url, dds_landing_url]:
    print(needs_update(landing_url, date_last_pulled))

print('-------')
date_last_pulled = '2020-08-01' # Example 2

for landing_url in [cdtfa_landing_url, dgs_landing_url, dds_landing_url]:
    print(needs_update(landing_url, date_last_pulled))

False
False
False
-------
True
False
True


## Pull datasets and print government building addresses listed

In [82]:
for data_url in [cdtfa_data_url, dgs_data_url, dds_data_url]:
    get_addresses(data_url)
    print('\n\n')

['1800 30th St', '12750 Center Court Drive South', '5901 Green Valley Circle', '1550 West Main St.', '2480 Hilborn Road', '8050 N Palm Ave', '505 N Brand Blvd', '16715 Von Karman Avenue', '1515 Clay St.', '35-900 Bob Hope Dr.', '2881 Churn Creek Rd.', '3737 Main St.', '3321 Power Inn Rd.', '450 N Street', '950 E. Blanco Road', '15015 Avenue of Science', '250 South Second St.', '25360 Magic Mountain Parkway', '50 D St.', '4820 McGrath St.', '1521 West Cameron Ave.', '120 N. La Salle St.', '1155 Dairy Ashford', '485 Lexington Avenue', '1030 Riverside Parkway', '3321 Power Inn Rd.']
['Suite 380', 'Ste. 400', 'Ste. 200', ' ', 'Suite 200', 'Ste 205', 'Ste. 700', 'Suite 200', 'Ste. 303', 'Ste. 280', 'Ste. B', 'Ste. 1000', 'Ste. 210', ' ', 'Ste. 202', 'Suite 200', ' ', 'Suite 330', 'Room 230', 'Ste. 260', 'Ste. 300', 'Ste. 1500', 'Ste. 550', 'Suite 400', 'Suite 125', 'Ste. 130']



['900 N Street', '914 Capitol Mall', '915 Capitol Mall', '31 E. Channel St.', '31 E. Channel St.', '744 P Street

# Sandbox
Everything below this is just the remnants from when I was testing things out and learning; I'm keeping them here for my own reference.

In [44]:
date_last_updated = find_date(cdtfa_landing_url)
date_last_updated

'2020-09-27'

In [45]:
date_last_pulled = str(date.today())
date_last_pulled

'2020-09-29'

In [21]:
cdtfa_url = 'http://data-cdtfa.opendata.arcgis.com/datasets/6b7ab4e55a2d463db9e902c2496c6e37_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D'
s = requests.get(cdtfa_url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

,X,Y,OBJECTID,doid,city,address1,address2,state,zip,phone,fax,directions,comments,type,DISTRICT_OF_LOC,OfficeType
0,-1.324950e+07,4.216651e+06,1,1,Bakersfield,1800 30th St,Suite 380,CA,93301-1922,(661) 395-2880,(661) 395-2588,"https://www.google.com/maps/dir//1800+30th+St,...",Bakersfield,in,DFB,Bakersfield
1,-1.314243e+07,4.011057e+06,2,6,Cerritos,12750 Center Court Drive South,Ste. 400,CA,90703-8594,(562) 356-1102,(562) 402-8503,https://www.google.com/maps/dir//12750+Center+...,Cerritos,in,AA,Cerritos
2,-1.317931e+07,4.026731e+06,3,2,Culver City,5901 Green Valley Circle,Ste. 200,CA,90230-6948,(310) 342-1000,(310) 342-1061,https://www.google.com/maps/dir//5901+Green+Va...,Culver City,in,AS,Culver City
3,-1.286559e+07,3.867842e+06,4,3,El Centro,1550 West Main St.,,CA,92243-2832,(760) 352-3431,(760) 352-8149,https://www.google.com/maps/dir//1550+W+Main+S...,El Centro,in,FHA,El Centro
4,-1.358663e+07,4.618437e+06,5,32,Fairfield,2480 Hilborn Road,Suite 200,CA,94534,(707) 427-4800,(707) 427-4815,https://www.google.com/maps/dir//2480+Hilborn+...,Fairfield,in,JHF,Fairfield


In [23]:
address_col_names = []
for x in df.columns:
    if 'address' in x.lower():
        address_col_names.append(x)
address_col_names

['address1', 'address2']

In [24]:
for address_col_name in address_col_names:
    print(address_col_name)
    print(list(df[address_col_name]))

address1
['1800 30th St', '12750 Center Court Drive South', '5901 Green Valley Circle', '1550 West Main St.', '2480 Hilborn Road', '8050 N Palm Ave', '505 N Brand Blvd', '16715 Von Karman Avenue', '1515 Clay St.', '35-900 Bob Hope Dr.', '2881 Churn Creek Rd.', '3737 Main St.', '3321 Power Inn Rd.', '450 N Street', '950 E. Blanco Road', '15015 Avenue of Science', '250 South Second St.', '25360 Magic Mountain Parkway', '50 D St.', '4820 McGrath St.', '1521 West Cameron Ave.', '120 N. La Salle St.', '1155 Dairy Ashford', '485 Lexington Avenue', '1030 Riverside Parkway', '3321 Power Inn Rd.']
address2
['Suite 380', 'Ste. 400', 'Ste. 200', ' ', 'Suite 200', 'Ste 205', 'Ste. 700', 'Suite 200', 'Ste. 303', 'Ste. 280', 'Ste. B', 'Ste. 1000', 'Ste. 210', ' ', 'Ste. 202', 'Suite 200', ' ', 'Suite 330', 'Room 230', 'Ste. 260', 'Ste. 300', 'Ste. 1500', 'Ste. 550', 'Suite 400', 'Suite 125', 'Ste. 130']


In [4]:
dgs_url = 'https://data.ca.gov/dataset/c5a3c577-3d2f-47fa-85f7-7aae00b2adc3/resource/def5d673-e39a-461d-bef1-a417579fd02e/download/dgs-conference-rooms.csv'
s = requests.get(dgs_url).content
dgs_data = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [9]:
dgs_data.head()

,RegionName,BuildingName,Address1,City,State,ZIP,RoomName,RoomComments,Contact,Telephone,...,VideoConferencing,ProjectionScreen,SpeakerSystem,Projector,TeleDataOutlet,WhiteBoard,VCRorDVD,TenantUse,StateAgencyUse,PublicUse
0,Capitol Historic Region,900 N Street,900 N Street,Sacramento,CA,95814,501,NaN,Jack Nolan,916-654-0033,...,N,Y,N,N,Y,Y,N,N,N,N
1,Capitol Historic Region,914 Capitol Mall,914 Capitol Mall,Sacramento,CA,95814,500,NaN,Jack Nolan,916-654-0033,...,N,Y,N,N,Y,N,Y,Y,N,N
2,Capitol Historic Region,915 Capitol Mall,915 Capitol Mall,Sacramento,CA,95814,587,NaN,Luis Hermosillo,916-653-3382,...,N,Y,N,N,Y,N,N,Y,N,N
3,Region II,31 E. Channel St.,31 E. Channel St.,Stockton,CA,95202,A-10,NaN,Building Manager,209-948-7730,...,N,Y,Y,N,N,N,N,Y,Y,N
4,Region II,31 E. Channel St.,31 E. Channel St.,Stockton,CA,95202,Auditorium,NaN,Building Manager,209-948-7730,...,N,Y,Y,N,N,N,N,Y,Y,N


In [56]:
address_col_names = []
for x in dgs_data.columns:
    if 'address' in x.lower():
        address_col_names.append(x)
address_col_names

['Address1']

In [59]:
for address_col_name in address_col_names:
    print(address_col_name)
    print(list(dgs_data[address_col_name]))

Address1
['900 N Street', '914 Capitol Mall', '915 Capitol Mall', '31 E. Channel St.', '31 E. Channel St.', '744 P Street (OB-9)', '744 P Street (OB-9)', '744 P Street (OB-9)', '744 P Street (OB-9)', '744 P Street (OB-9)', '744 P Street (OB-9)', '744 P Street (OB-9)', '744 P Street (OB-9)', '744 P Street (OB-9)', '744 P Street (OB-9)', '1600 9th St.', '1600 9th St.', '901 P St.', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '1516 9th Street', '2440 Main Street', '2135 Civic Center Drive', '1500 11th St.', '1500 11th St.', '9645 Butterfield Way', '9645 Butterfield 